# Generation of fsusie results
The notebook is to documents the intergration of results from different runs of haQTL that results from the ongoing development of susiF package.
1. The first runs has dummy CS and with fail purity calculation
2. The second runs stop prematurely due to mem issues
3. The third runs contains CS that are not discovered due to too low a upper bound of discoverable CS
4. The tads with less than 16 phenotypes are removed.


In [1]:
options(repr.plot.width =30, repr.plot.height = 20)

## Fsusie Commands

In [ ]:
#### mQTL

In [ ]:




sos run pipeline/SuSiE.ipynb uni_fsusie \
--genoFile /mnt/vast/hpc/csg/molecular_phenotype_calling/eqtl/output/dlpfc_tad_region_list_plink_files/ROSMAP_NIA_WGS.leftnorm.filtered.filtered.plink_files_list.txt  \
--cwd output/f_susie_tad_meQTL_pos_resume/ --name meQTL \
--region-list dlpfc_tad_list_resume_me \
--phenoFile methy_phenotype_list \
--covFile methy_covar_list --prior mixture_normal_per_scale -J 20 --job_size 1 -c csg2.yml -q csg2 -s build  --mem 80G --walltime 48h --max_SNP_EM 100

## Data loading

In [1]:
library("dplyr")
library("readr")
library("purrr")
library("gwasrapidd")


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘gwasrapidd’


The following object is masked from ‘package:dplyr’:

    n




In [4]:
library(stringr)

In [ ]:
eQTL = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/eqtl/output/susie_per_gene_tad/whole_genome_cs.eqtl",delim = "\t")

## haQTL (0.12.0.3)
This step is to accomodate the different runs of haQTL that results from the ongoing development of susiF package.

In [67]:
haQTL = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/f_susie_tad_haQTL_pos_check_pure_2/result/haQTL.all_variants.tsv",delim = "\t")


Rows: 5795595 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


### Remove dummy CS (0.12.0.4)

In [68]:
haQTL_purity = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/f_susie_tad_haQTL_pos_check_pure_3/haQTL.all_variants.tsv",delim = "\t")
dummy_tad = haQTL_purity%>%filter(abs(purity) < 0.5 )%>%count(molecular_trait_id,cs_order)%>%arrange(n)%>%pull(molecular_trait_id)

Rows: 2558538 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [69]:
haQTL_pure = haQTL%>%filter(!molecular_trait_id %in% dummy_tad)

### Add resumed runs (0.12.0.7)
The results from this run use a algorithim that limited the number of wavelet coefficience to be used in the fsusie analysis

In [42]:
haQTL_resume = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/f_susie_tad_haQTL_pos_resume/result/haQTL.all_variants.tsv")

Rows: 547080 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [73]:
haQTL_resume_pure = haQTL_resume%>%filter((abs(purity) > 0.5 ))

In [74]:
haQTL_pure_completed = bind_rows(haQTL_pure%>%filter(cs_order != 0,!molecular_trait_id %in% haQTL_resume_pure$molecular_trait_id ),haQTL_resume_pure%>%filter(cs_order != 0 ))

In [75]:
haQTL_pure_completed%>%filter(abs(purity) < 0.5)

variants,snps_index,chromosome,position,ref,alt,cs_order,pip,molecular_trait_id,finemapped_region_start,finemapped_region_end,purity,is_dummy,effect_peak_pos
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>


### Add unlimtied runs (0.12.0.3)
The result from this run use max_L = 20 instead of 10 to allows for additional results

In [76]:
haQTL_unlimited = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/f_susie_tad_haQTL_pos_limited_2//result/haQTL.all_variants.tsv")

Rows: 88018 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [77]:
haQTL_pure_completed_unlimited = bind_rows(haQTL_pure_completed%>%filter(!molecular_trait_id %in% haQTL_unlimited$molecular_trait_id ),haQTL_unlimited%>%filter(cs_order != 0))

In [78]:
haQTL_pure_completed_unlimited%>%nrow

[1] 29043

In [80]:
haQTL_pure_completed_unlimited%>%write_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/haQTL_pure_completed_unlimited.tsv","\t")

## mQTL (0.12.0.3)

In [109]:
meQTL = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/f_susie_tad_meQTL_pos_2/result/meQTL.all_variants.tsv",delim = "\t")


Rows: 5922387 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


### Remove dummy CS (0.12.0.4)

In [110]:
meQTL_purity = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/f_susie_tad_meQTL_pos_check_pure/meQTL.all_variants.tsv",delim = "\t")
dummy_tad = meQTL_purity%>%filter(abs(purity) < 0.5 )%>%count(molecular_trait_id,cs_order)%>%arrange(n)%>%pull(molecular_trait_id)

Rows: 313834 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [111]:
meQTL_pure = meQTL%>%filter(!molecular_trait_id %in% dummy_tad)

### Add resumed runs (0.12.0.7)
The results from this run use a algorithim that limited the number of wavelet coefficience to be used in the fsusie analysis

In [112]:
meQTL_resume = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/f_susie_tad_meQTL_pos_resume//result/meQTL.all_variants.tsv")

Rows: 394394 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [113]:
meQTL_resume_pure = meQTL_resume%>%filter(abs(purity) < 0.5 )

In [114]:
meQTL_pure_completed = bind_rows(meQTL_pure%>%filter(cs_order != 0 ),meQTL_resume_pure%>%filter(cs_order != 0 ))

### Add unlimtied runs (0.12.0.3)
The result from this run use max_L = 20 instead of 10 to allows for additional results

In [115]:
meQTL_unlimited = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/f_susie_tad_meQTL_pos_limited_2///result/meQTL.all_variants.tsv")

Rows: 2008778 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [116]:
meQTL_pure_completed_unlimited = bind_rows(meQTL_pure_completed%>%filter(!molecular_trait_id %in% meQTL_unlimited$molecular_trait_id ),meQTL_unlimited%>%filter(cs_order != 0))

In [117]:
meQTL_pure_completed_unlimited%>%nrow

[1] 65751

In [118]:
meQTL_pure_completed_unlimited%>%write_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/meQTL_pure_completed_unlimited.tsv","\t")

In [101]:
a = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/meQTL_pure_completed_unlimited.tsv")

Rows: 34568 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [102]:
a

variants,snps_index,chromosome,position,ref,alt,cs_order,pip,molecular_trait_id,finemapped_region_start,finemapped_region_end,purity,is_dummy,effect_peak_pos
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr12:117363680_C_A,437,12,117363680,C,A,1,0.9999999107,tad1069,117200000,117840000,1.0000000,0,117578659
chr12:117438669_A_G,645,12,117438669,A,G,4,0.2280806278,tad1069,117200000,117840000,0.9337547,0,117316023
chr12:117439202_A_G,646,12,117439202,A,G,4,0.1487677370,tad1069,117200000,117840000,0.9337547,0,117316023
chr12:117441178_G_A,651,12,117441178,G,A,4,0.1590309865,tad1069,117200000,117840000,0.9337547,0,117316023
chr12:117441215_A_G,652,12,117441215,A,G,4,0.0397854995,tad1069,117200000,117840000,0.9337547,0,117316023
chr12:117441519_T_C,654,12,117441519,T,C,4,0.0397854995,tad1069,117200000,117840000,0.9337547,0,117316023
chr12:117441541_A_G,655,12,117441541,A,G,4,0.0397854995,tad1069,117200000,117840000,0.9337547,0,117316023
chr12:117442206_A_G,656,12,117442206,A,G,4,0.0209240945,tad1069,117200000,117840000,0.9337547,0,117316023
chr12:117443459_G_A,661,12,117443459,G,A,4,0.0116282333,tad1069,117200000,117840000,0.9337547,0,117316023


In [108]:
meQTL_pure%>%filter(molecular_trait_id == "tad100", cs_order != 0)

variants,snps_index,chromosome,position,ref,alt,cs_order,pip,molecular_trait_id,finemapped_region_start,finemapped_region_end,purity,is_dummy,effect_peak_pos
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1:206430185_G_A,23,1,206430185,G,A,1,0.99608899,tad100,206400000,207560000,1.0000000,0,206428266
chr1:206503837_C_T,207,1,206503837,C,T,5,0.09347794,tad100,206400000,207560000,0.9914452,0,206548725
chr1:206503888_T_G,208,1,206503888,T,G,5,0.18938234,tad100,206400000,207560000,0.9914452,0,206548725
chr1:206504303_A_C,209,1,206504303,A,C,5,0.18938234,tad100,206400000,207560000,0.9914452,0,206548725
chr1:206505665_C_A,214,1,206505665,C,A,5,0.29398114,tad100,206400000,207560000,0.9914452,0,206548725
chr1:206509762_G_T,222,1,206509762,G,T,5,0.10410848,tad100,206400000,207560000,0.9914452,0,206548725
chr1:206512396_C_T,226,1,206512396,C,T,5,0.10410848,tad100,206400000,207560000,0.9914452,0,206548725
chr1:206645407_GTATCCTAGACTTCCTTTTATT_G,542,1,206645407,GTATCCTAGACTTCCTTTTATT,G,8,0.98439225,tad100,206400000,207560000,1.0000000,0,206752833
chr1:206674795_T_C,616,1,206674795,T,C,3,0.54075482,tad100,206400000,207560000,-0.9289792,1,206760075


In [1]:
meQTL_pure

ERROR: Error in eval(expr, envir, enclos): object 'meQTL_pure' not found


## Reselect result with read > 16
### haQTL

In [2]:
library("tidyverse")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0     ✔ purrr   0.3.4
✔ tibble  3.1.8     ✔ dplyr   1.0.7
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [4]:
tad_list =  read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/dlpfc_tad_list")
ha = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/ha_peak_list")
m = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/methylation_cpg_list")

Rows: 1469 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): #chr, tad_index
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
ha = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/ha_peak_list")
m = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/methylation_cpg_list")

Rows: 92401 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): #chr, ID
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
ha_tad = inner_join(ha,tad_list,by = "#chr")

In [10]:
ha_tad = inner_join(ha,tad_list,by = "#chr")%>%filter(start.x > start.y,end.x < end.y  )
ha_tad_16 = ha_tad%>%count(tad_index)%>%filter(n >= 16)

### mQTL

In [16]:
m = read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/methylation_cpg_list")

Rows: 395070 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): #chr, ID
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [18]:
m_tad = inner_join(m,tad_list,by = "#chr")

In [20]:
m_tad = m_tad%>%filter(start.x > start.y,end.x < end.y  )

In [21]:
m_tad_16 = m_tad%>%count(tad_index)%>%filter(n >= 16)

In [22]:
m_tad_16

tad_index,n
<chr>,<int>
tad1,4221
tad10,122
tad100,277
tad1000,315
tad1001,127
tad1002,123
tad1003,423
tad1004,422
tad1005,131


In [24]:
meQTL_pure_completed_unlimited= read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/meQTL_pure_completed_unlimited.tsv","\t")
haQTL_pure_completed_unlimited= read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/haQTL_pure_completed_unlimited.tsv","\t")

Rows: 65751 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 29043 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (4): variants, ref, alt, molecular_trait_id
dbl (10): snps_index, chromosome, position, cs_order, pip, finemapped_region...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [25]:
meQTL_pure_completed_unlimited_16 = meQTL_pure_completed_unlimited%>%filter(molecular_trait_id %in% m_tad_16$tad_index)

In [26]:
haQTL_pure_completed_unlimited_16 = haQTL_pure_completed_unlimited%>%filter(molecular_trait_id %in% ha_tad_16$tad_index)

In [31]:
meQTL_pure_completed_unlimited_16 %>%write_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/meQTL_pure_completed_unlimited_16.tsv","\t")
haQTL_pure_completed_unlimited_16%>%write_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/QTL_fine_mapping/output/haQTL_pure_completed_unlimited_16.tsv","\t")

In [29]:
m_tad_16%>%filter(tad_index %in% c("tad808","tad997","tad435"))

tad_index,n
<chr>,<int>
tad435,119
tad808,846
tad997,50


In [30]:
ha_tad_16%>%filter(tad_index %in% c("tad808","tad997","tad435"))

tad_index,n
<chr>,<int>
tad435,46
tad808,135
tad997,75


In [34]:
m_tad_16

tad_index,n
<chr>,<int>
tad1,4221
tad10,122
tad100,277
tad1000,315
tad1001,127
tad1002,123
tad1003,423
tad1004,422
tad1005,131


Rows: 1381 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): #chr, index
dbl (2): start, end

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [5]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [7]:
a = readr::read_delim("/mnt/vast/hpc/csg/molecular_phenotype_calling/TAD_TADB/generalized_TADB_0706.tsv")
a%>%mutate(diff = end - start)%>%arrange(diff)%>%filter(diff < 2000000)

#chr,start,end,index,diff
<chr>,<dbl>,<dbl>,<chr>,<dbl>
chr11,10360000,10760000,TADB186,400000
chr20,19680000,20080000,TADB703,400000
chr9,69920000,70320000,TADB1297,400000
chr7,22080000,22520000,TADB1136,440000
chr11,116760000,117240000,TADB239,480000
chr12,12320000,12800000,TADB258,480000
chr3,111600000,112080000,TADB808,480000
chr8,18880000,19360000,TADB1212,480000
chr9,70240000,70720000,TADB1298,480000


In [ ]:
a = readr::read_delim("~/test_mem/Cortex-EUR-AFR-noENA-100pcs-trans-chr6-sorted.bgz.txt")